In [7]:
from scipy.linalg import expm
import qorelib as q
import numpy as np
from numpy import ndarray, matrix, complexfloating
from tqdm import tqdm
from scipy.optimize import leastsq
import plotly.graph_objects as go


In [8]:
OmegaQ: float = 2 * np.pi * 5000
OmegaD: float = 2 * np.pi * 5000 - 2 * np.pi * 1  # 这个频率是角频率
# a: complexfloating = 1 / np.sqrt(2) * 1
# b: complexfloating = 1 / np.sqrt(2)
a: complexfloating = 0
b: complexfloating = 1
# b: complexfloating = (100j) / np.sqrt(10000)
A: float = 30 * 2 * np.pi * 1
PhiD: float = np.pi / 2 * 0


initialState:matrix = matrix([[a], [b]])


In [9]:
def HamiltonianWithouDrive2X2(omegaQ: float, omegaD: float, phiD: float, A: float):
    delta: float = omegaD - omegaQ
    H: matrix = matrix([[0, 0], [0, -delta]])
    return H


def HamiltonianWithDrive2X2X(omegaQ: float, omegaD: float, phiD: float, A: float):
    delta: float = omegaD - omegaQ
    H: matrix = matrix([[0, 0.5 * A * np.exp(1j * phiD)],
                       [0.5 * A * np.exp(-1j * phiD), -delta]])
    return H


In [10]:
data = []
stateData = []
tao = 0.3
for t in range(0, int(tao * 1000)):
    td = t / 1000
    currentState: matrix = q.ComputeState(initialState, td, HamiltonianWithDrive2X2X(OmegaQ, OmegaD, PhiD, A))
    stateData.append(currentState.T.tolist())
    data.append([td, q.ComputeStateProbability(currentState, 0)])
ndData = np.array(data)
ndStateData = np.array(stateData)

fig = go.Figure()

fig.add_trace(go.Scatter(x=ndData[:, 0], y=ndData[:, 1], mode='markers'))

fig.update_yaxes(
    range=[0, 1.0],
    dtick=0.5,
)


fig.show()

In [11]:
data: ndarray = np.array([[0, 0]])
stateData: ndarray = np.array([[0, 0]])
τ: float = 0.3
precision: int = 1000
for t in tqdm(range(0, int(τ * precision)), desc="Computing ... ", colour="#12c462"):
    td = t / precision
    currentState: matrix = q.ComputeState(
        initialState, td, HamiltonianWithDrive2X2X(OmegaQ, OmegaD, PhiD, A))
    if t == 0:
        stateData[0] = np.array(currentState.T)
    else:
        stateData = np.r_[stateData, currentState.T]
    if t == 0:
        data[0] = np.array([td, q.ComputeStateProbability(currentState, 0)])
    else:
        data = np.r_[data, np.array(
            [[td, q.ComputeStateProbability(currentState, 0)]])]

Computing ... :   0%|          | 0/300 [00:00<?, ?it/s]C:\Users\seanw\AppData\Local\Temp\ipykernel_17836\1819638194.py:10: ComplexWarning:

Casting complex values to real discards the imaginary part

Computing ... : 100%|██████████| 300/300 [00:00<00:00, 2946.30it/s]


In [12]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data[:, 0], y=data[:, 1], mode='markers'))

fig.update_yaxes(
    range=[0, 1.0],
    dtick=0.5,
)


fig.show()